In [16]:
import pandas as pd
import os
from os import path
import glob
import numpy as np
from nltk.corpus import stopwords
import nltk
from collections import Counter
import dask.dataframe as dd
import math

In [174]:
stopz = [word.split() for word in open("C:\\Users\\Ruben\\Documents\\Artikelen\\Joris\\stopwords-nl.txt", 'r', encoding = "utf-8").readlines()]

In [2]:
## Import Master DF
os.chdir("C://Users//Ruben//Documents//Scriptie//Data//bigrams//output")
df = dd.read_csv('total-bigrams.csv', dtype='str')

In [57]:
## Load Buitenlandsche Bigrams
bgdf = pd.read_csv('C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data//bigrams_buitenlandsche.csv', encoding = 'utf-8')

In [77]:
def GetListWords(words_year):
    dwy = dict()
    
    for i in list(range(1,100)):
        tdf = bgdf.iloc[:,[0,i]]
        y = tdf.columns[1]
        tdf = tdf.sort_values([y], ascending = False)
        tdf = tdf.reset_index(drop = True)
        tdf = tdf.ngram[0:words_year-1]
        
        dwy.update({y:list(tdf)})
    return dwy

In [84]:
total_list = GetListWords(20).values()
total_list = list(set([item for sublist in total_list for item in sublist]))
total_list = [l for l in total_list if len(l) > 2]

In [108]:
def GetBigrams(input_list):
    subset = df[df['ngram'].str.split().str[1].isin(input_list)].compute()
    subset = subset.reset_index(drop=True)
    subset['count'] = subset['count'].astype(float)
    subset = subset.groupby(['ngram','year']).agg('sum')
    subset = subset.stack().reset_index().drop('level_2', axis = 1)
    subset.columns = ['ngram', 'year', 'count']
    subset = subset.dropna()
    subset = subset.pivot(index='ngram', columns='year', values='count')
    subset = subset.fillna(0)
    subset.columns = subset.columns.astype(str)
    
    return subset

In [109]:
dfr = GetBigrams(total_list)

In [177]:
def GetListAdjectives(input_df, freq_thres):
    dwy = dict()
    
    for i in list(range(1840,1914)):
        yl = list()
        tdf = input_df[input_df[str(i)] > freq_thres]
        for key,value in Counter(list(tdf.ngram.str.split().str[0])).items():
            if value > 10 and tag(key)[0][1] == 'JJ' and len(key) > 2 and key not in stopz:
                yl.append(key)
            
        
        dwy.update({i:yl})
    return dwy

In [179]:
lad = GetListAdjectives(dfr, 5)

In [180]:
lad_df = pd.DataFrame()

for key, value in lad.items():
    tmp = pd.DataFrame(value)
    tmp['year'] = key
    lad_df = lad_df.append(tmp)

In [181]:
lad_df

,0,year
0,buitenlandsche,1840
1,dezelve,1840
2,engelsche,1840
3,fransche,1840
4,geheel,1840
5,nieuwe,1840
6,vreemde,1840
0,buitenlandsche,1841
1,dezelve,1841
2,eenige,1841
